In [1]:
## UTILITIES

import re

def check_distinct(rdd):
  ''' check if all words in dict_ are distinct '''
  tweet_ids = rdd.map(lambda x: x[0])
  total_count = tweet_ids.count()
  distinct_count = tweet_ids.distinct().count()
  if total_count == distinct_count:
    print("All tweets are distinct")
  else:
    print("All tweets are NOT distinct")
  
  
def process(text):
  ''' remove punctuation, make all lowercase, split'''
  pattern = re.compile('[^\w ]+')
  text = pattern.sub('', text) # remove punctuation
  text = text.lower() # make lowercase
  return text.split(" ")

  
def binary_search(val, arr):
  ''' binary search for val in arr
  return index if exists, otherwise -1
  '''
  left, right = 0, len(arr) - 1
  while left <= right:
      mid = left + (right - left) // 2
      mid_val = arr[mid]
      if val < mid_val:
          right = mid - 1
      elif val > mid_val:
          left = mid + 1
      else:
          return mid
  return -1


def clean_word(word):
  ''' remove non-alphanumeric and make lowercase '''
  pattern = re.compile('[\W]+')
  return pattern.sub('', word.lower()) 


# get list of legitimate english words
ENGLISH_WORDS = sorted(sc.textFile("/FileStore/tables/words.txt").map(clean_word).collect())


def is_legit(word):
  ''' check if word is a legitimate english word '''
  if binary_search(word[0], ENGLISH_WORDS) >= 0:
    return True
  return False

In [2]:
def vocab_ratio(username, tweets):
  ''' Analyzes tweets of user to determine the vocab size per 10k words '''  
  SAMPLE_SIZE = 5000
#   username, tweets = user
  rdd = sc.parallelize(tweets.asDict().items())
  print(f"processing tweets of: {username}")
#   check_distinct(rdd)
  # extract only text, and filter out retweets
  tweets_rdd = rdd.map(lambda x: x[1]).filter(lambda tweet: tweet[:2] != "RT")
  # separate words, filter out improper words
  words_rdd = tweets_rdd.flatMap(process).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b).filter(is_legit).flatMap(lambda x: [x[0]] * x[1])
  # randomly sampling 10k words
  total_count = words_rdd.count()
  if total_count < SAMPLE_SIZE:
    print("not enough words")
    return
  sampled_rdd = words_rdd.sample(withReplacement=False, fraction=SAMPLE_SIZE/total_count)
  # count frequency of each word
  freq_rdd = sampled_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
  distinct_count = freq_rdd.count()
  sampled_count = sampled_rdd.count()
  ratio = distinct_count / sampled_count
  print(f"{distinct_count} out of {sampled_count} words. Ratio: {ratio}")
  return ratio
  

In [3]:
file_location = "/FileStore/tables/all_tweets.json"
rdd = spark.read.option("multiline", "true").json(file_location).rdd.flatMap(lambda x: list(x.asDict().items()))

In [4]:
ratio_dict = {}
for user in rdd.collect():
  username, tweets = user
  ratio = vocab_ratio(username, tweets)
  ratio_dict[username] = ratio

processing tweets of: ArmanRo36332524
1193 out of 5016 words. Ratio: 0.2378389154704944
processing tweets of: Bala06809628
not enough words
processing tweets of: BeIshita
1550 out of 4998 words. Ratio: 0.31012404961984796
processing tweets of: BeingKalptaru
not enough words
processing tweets of: BichAssClaudia
1229 out of 5000 words. Ratio: 0.2458
processing tweets of: BieberPrasad
not enough words
processing tweets of: BottomAmethyst
1322 out of 4955 words. Ratio: 0.26680121089808273
processing tweets of: BryanDMcNally
1473 out of 5037 words. Ratio: 0.29243597379392494
processing tweets of: CrownTheGrind
not enough words
processing tweets of: DIAMONDRED05
not enough words
processing tweets of: Goodweeed_m
1167 out of 4982 words. Ratio: 0.23424327579285428
processing tweets of: JeffreyLuscombe
1703 out of 5008 words. Ratio: 0.340055910543131
processing tweets of: Kayla_Patriceee
1085 out of 4942 words. Ratio: 0.21954674220963172
processing tweets of: KieranK13ran
not enough words
processing tweets of: LivingSno
1674 out of 4899 words. Ratio: 0.3417023882424985
processing tweets of: MeMuddaFuckka
1145 out of 4944 words. Ratio: 0.23159385113268607
processing tweets of: Memetherapy1
1656 out of 5019 words. Ratio: 0.3299462044231919
processing tweets of: Mifaunuagbo
1372 out of 5040 words. Ratio: 0.2722222222222222
processing tweets of: MirandaPokorsk
not enough words
processing tweets of: Morecently
not enough words
processing tweets of: NWellyf
1576 out of 5021 words. Ratio: 0.31388169687313283
processing tweets of: NoNeedToBeKnow2
not enough words
processing tweets of: OfficialNYCG
not enough words
processing tweets of: Orangezipple1
1415 out of 4947 words. Ratio: 0.2860319385486153
processing tweets of: OtbVontee
1007 out of 5019 words. Ratio: 0.20063757720661488
processing tweets of: PatriotTrump77
not enough words
processing tweets of: PhDholda
1191 out of 5073 words. Ratio: 0.23477232406859846
processing tweets of: Preeti86266373
not enough words
processing tweets of: RJKINGJOHNSON
not enough words
processing tweets of: RabinaMagar13
not enough words
processing tweets of: Riarna_K
not enough words
processing tweets of: RobWils88348869
not enough words
processing tweets of: Royal_SeokJin
not enough words
processing tweets of: Sam_Vinokor
1869 out of 5030 words. Ratio: 0.3715705765407555
processing tweets of: Sh3n1gam1
not enough words
processing tweets of: Shamiso_
1261 out of 4961 words. Ratio: 0.2541826244708728
processing tweets of: TPRISl
not enough words
processing tweets of: TRUMPGETGO
not enough words
processing tweets of: TaskForceStaff
917 out of 5005 words. Ratio: 0.18321678321678322
processing tweets of: TomasHearty
1432 out of 5025 words. Ratio: 0.28497512437810946
processing tweets of: TrechxTV
1137 out of 4941 words. Ratio: 0.23011536126290225
processing tweets of: Vwinterbear_95
not enough words
processing tweets of: WarzoneSpace
1264 out of 5050 words. Ratio: 0.2502970297029703
processing tweets of: Yo_ShiSmooth
1112 out of 5033 words. Ratio: 0.2209417842241208
processing tweets of: alyaJoooooo
not enough words
processing tweets of: anthonylauro_
1463 out of 5024 words. Ratio: 0.29120222929936307
processing tweets of: bertus161057
not enough words
processing tweets of: btssowooju
1190 out of 5049 words. Ratio: 0.2356902356902357
processing tweets of: chaloxcervantes
1240 out of 5046 words. Ratio: 0.24573919936583433
processing tweets of: chamomicrossing
not enough words
processing tweets of: chan_minsshi
not enough words
processing tweets of: chandlerblackEP
not enough words
processing tweets of: choconaut_
1393 out of 4894 words. Ratio: 0.2846342460155292
processing tweets of: colleenmerrells
1117 out of 5101 words. Ratio: 0.21897667124093315
processing tweets of: connornuts
1497 out of 4998 words. Ratio: 0.2995198079231693
processing tweets of: conser_usa
1101 out of 5105 words. Ratio: 0.2156709108716944
processing tweets of: domcrofts
1504 out of 4880 words. Ratio: 0.3081967213114754
processing tweets of: 

In [5]:
ratio_dict

In [6]:
import json

write_file_location = "/dbfs/FileStore/output/ratio_dict.txt"
with open(write_file_location, 'w') as f:
  f.write(json.dumps(ratio_dict))

In [7]:
## reading and writing file
# file location: "/dbfs/FileStore/output/test_file.txt"
# url to download file: https://community.cloud.databricks.com/files/output/ratio_dict.txt?o=8290779921351936